# Simulating and Storing Brain Network Activity for Multiple "Patients"

This Python script leverages the `neurolib` library to simulate brain activity using the **Wilson-Cowan (WC) model** across a whole-brain network. The primary goal is to generate simulated neural signals for a cohort of 200 "patients," each assigned to one of several predefined neurological conditions (e.g., healthy, coma, brain death). The raw and binarized (thresholded) activity for each simulated patient is then saved to disk.

---

## Script Overview

The script performs the following main steps:

1.  **Configuration**: Sets up simulation parameters like duration and binarization threshold.
2.  **Condition Definition**: Defines a set of distinct neurological "conditions," each with its own specific set of Wilson-Cowan model parameters. These parameters are chosen to approximate the neurobiological characteristics of each state.
3.  **Data Directory Setup**: Ensures a directory exists to store the output data.
4.  **HCP Dataset Loading**: Loads structural connectivity (`Cmat`) and distance (`Dmat`) matrices from the Human Connectome Project (HCP) dataset, which are crucial for simulating a whole-brain network.
5.  **Simulation Loop**: Iterates through each defined condition, running multiple simulations per condition to reach the target of 200 patients.
6.  **Signal Processing**: For each simulation, it extracts the excitatory neural activity and binarizes it based on a statistical threshold.
7.  **Data Saving**: Saves both the raw and binarized simulated signals for each patient into separate `.pkl` (pickle) files.

---

## Key Components and Features

### 1. General Configuration

* `duration`: Sets the total simulation time in milliseconds (e.g., 20,000 ms or 20 seconds).
* `threshold_multiplier`: Used to calculate a binarization threshold for activity. Signals exceeding `mean + threshold_multiplier * std` are considered "active" (1), otherwise "inactive" (0).

### 2. `subject_conditions` Dictionary

This is the core of simulating different patient types. It's a dictionary where each key represents a neurological condition (e.g., `"Healthy_Critical"`, `"Coma"`), and its value is another dictionary containing specific **Wilson-Cowan model parameters** for that condition.

* **Parameter Variation**: Parameters like `exc_ext` (external excitatory input), `K_gl` (global coupling), `sigma_ou` (noise level), `tau_exc`/`tau_inh` (time constants), and `a_exc`/`a_inh` / `c_exc`/`c_inh` (sigmoid gains and intrinsic weights) are carefully tuned for each condition to mimic different brain states.
* **Filename Friendly Names**: The keys are named using underscores (e.g., `Healthy_Critical`, `Brain_Death`) to be suitable for use in filenames.

### 3. Data Storage (`pathlib` and `pickle`)

* **`pathlib.Path`**: Used for robust and cross-platform handling of file paths, making it easy to create directories and construct filenames.
* **`data_dir = Path("../data")`**: Defines the output directory as `../data` (one level up from the script's location). The `.mkdir(parents=True, exist_ok=True)` ensures the directory exists, creating parent directories if necessary.
* **`pickle`**: The `pickle` module is used to serialize (save) Python objects (like NumPy arrays representing the simulated signals) into `.pkl` binary files. This is efficient for storing numerical data.
* **Filename Convention**: Each patient's data is saved in two files following the convention: `patient_XXX_TYPE_raw.pkl` and `patient_XXX_TYPE_binarized.pkl`, where `XXX` is a zero-padded patient ID (e.g., `001`) and `TYPE` is the condition key (e.g., `healthy_critical`).

### 4. Simulation Loop and Patient Distribution

* **`num_patients = 200`**: Sets the total number of patients to simulate.
* **Equal Distribution**: The script calculates `patients_per_condition` to distribute the total patients as equally as possible among the defined conditions. It also handles any remainder patients by assigning them to the first conditions in the dictionary.
* **Model Instantiation**: For each simulation, a new `wc.WCModel` is created, crucially initialized with the `Cmat` and `Dmat` to enable whole-brain network dynamics.
* **Parameter Assignment**: The condition-specific parameters are assigned to `model.params` before running each simulation.

### 5. Signal Binarization

* After each simulation, the raw excitatory activity (`exc_raw`) is binarized channel by channel.
* The threshold for binarization is calculated dynamically for **each channel** as `mean_activity_per_channel + threshold_multiplier * std_activity_per_channel`. This adapts to the varying activity levels across different brain regions and conditions.
* A check `if std_i == 0:` is included to prevent division by zero errors if a channel has no variance (e.g., completely flat activity in "Brain Death").

---

This script provides a powerful framework for generating synthetic neuroimaging data that can be used for further analysis, such as studying avalanche dynamics, functional connectivity, or training machine learning models on different neurological states.

In [4]:
import neurolib.models.wc as wc
from neurolib.utils.loadData import Dataset
import numpy as np
import matplotlib.pyplot as plt # Keep import, but plotting code will be commented
import pickle
from pathlib import Path

# -------------------------------
# General Configuration
# -------------------------------
duration = 20 * 1000  # 20 seconds
threshold_multiplier = 2.0 # Multiplier for the binarization threshold

# --- Definition of subject conditions with their parameters ---
# NOTE: The values are EXAMPLES based on our discussion.
# Precise tuning requires research and data validation.
subject_conditions = {
    "Healthy_Critical": { # Changed name to be filesystem friendly
        'exc_ext': 0.65,
        'K_gl': 3.15,
        'sigma_ou': 0.14,
        'signalV': 0,
        'a_exc': 1.0, 'a_inh': 1.0,
        'tau_exc': 10.0, 'tau_inh': 10.0,
        'c_exc': 10.0, 'c_inh': 10.0,
        'plot_lim_title': "Healthy (Critical)" # This will not be used, but kept for consistency
    },
    "Coma": {
        'exc_ext': 0.15,
        'K_gl': 2.8,
        'sigma_ou': 0.05,
        'signalV': 0,
        'a_exc': 1.0, 'a_inh': 1.0,
        'tau_exc': 15.0, 'tau_inh': 15.0,
        'c_exc': 10.0, 'c_inh': 10.0,
        'plot_lim_title': "Coma (Reduced Global Activity)"
    },
    "Brain_Death": { # Changed name to be filesystem friendly
        'exc_ext': 0.01,
        'K_gl': 0.1,
        'sigma_ou': 0.001,
        'signalV': 0,
        'a_exc': 0.1, 'a_inh': 0.1,
        'tau_exc': 20.0, 'tau_inh': 20.0,
        'c_exc': 10.0, 'c_inh': 10.0,
        'plot_lim_title': "Brain Death (Absence of Function)"
    },
    "Induced_Coma_Propofol": { # Changed name to be filesystem friendly
        'exc_ext': 0.5,
        'K_gl': 3.15,
        'sigma_ou': 0.1,
        'signalV': 0,
        'a_exc': 1.0, 'a_inh': 1.0,
        'tau_exc': 10.0, 'tau_inh': 10.0,
        'c_exc': 10.0,
        'c_inh': 14.0,
        'plot_lim_title': "Induced Coma (Increased Inhibition)"
    },
    "Intellectual_Disability_Hyperexcitability": { # Changed name to be filesystem friendly
        'exc_ext': 0.75,
        'K_gl': 3.5,
        'sigma_ou': 0.14,
        'signalV': 0,
        'a_exc': 1.2, 'a_inh': 0.8,
        'tau_exc': 10.0, 'tau_inh': 10.0,
        'c_exc': 11.0, 'c_inh': 9.0,
        'plot_lim_title': "Intellectual Disability (e.g., Hyperexcitability)"
    }
}

# -------------------------------
# Create data directory
# -------------------------------
data_dir = Path("../data")
data_dir.mkdir(parents=True, exist_ok=True)
print(f"Data directory created/ensured at: {data_dir.resolve()}")

# -------------------------------
# Load dataset with Cmat and Dmat
# -------------------------------
print("Loading 'hcp' dataset...")
ds = Dataset("hcp")
Cmat = ds.Cmat
Dmat = ds.Dmat
n_channels = Cmat.shape[0] # This ensures all 80 channels are used
print(f"'hcp' dataset loaded with {n_channels} channels.")

# -------------------------------
# Simulation and Saving
# -------------------------------
num_patients = 200
num_conditions = len(subject_conditions)
patients_per_condition = num_patients // num_conditions
# Handle remainder if num_patients is not perfectly divisible
remainder_patients = num_patients % num_conditions

patient_id_counter = 1
for condition_idx, (condition_key, condition_params) in enumerate(subject_conditions.items()):
    print(f"\n--- Simulating {patients_per_condition} patients for condition: {condition_key} ---")

    current_patients_to_simulate = patients_per_condition
    if condition_idx < remainder_patients: # Distribute remainder among first conditions
        current_patients_to_simulate += 1

    for _ in range(current_patients_to_simulate):
        patient_id_str = f"{patient_id_counter:03d}" # Format as 001, 002, etc.

        print(f"  Simulating patient {patient_id_str} ({condition_key})...")

        # Create model
        model = wc.WCModel(Cmat=Cmat, Dmat=Dmat)
        model.duration = duration

        # Assign parameters for the current condition
        model.params['exc_ext'] = condition_params['exc_ext']
        model.params['sigma_ou'] = condition_params['sigma_ou']
        model.params['K_gl'] = condition_params['K_gl']
        model.params['signalV'] = condition_params['signalV']
        model.params['a_exc'] = condition_params['a_exc']
        model.params['a_inh'] = condition_params['a_inh']
        model.params['tau_exc'] = condition_params['tau_exc']
        model.params['tau_inh'] = condition_params['tau_inh']
        model.params['c_exc'] = condition_params['c_exc']
        model.params['c_inh'] = condition_params['c_inh']

        model.run()
        # time = model.outputs['t'] # No longer needed if not plotting
        exc_raw = model.outputs['exc']  # (n_channels, t)

        # Binarization per channel
        binary_exc = np.zeros_like(exc_raw, dtype=int)
        # thresholds_per_channel = [] # No longer needed if not plotting

        for i in range(n_channels): # This loop iterates over all 80 channels
            mean_i = np.mean(exc_raw[i])
            std_i = np.std(exc_raw[i])
            # Ensure std_i is not zero to avoid division by zero (e.g., in Brain Death)
            if std_i == 0:
                threshold = mean_i # If no variance, threshold at mean (effectively all 0 or all 1 if >0)
            else:
                threshold = mean_i + threshold_multiplier * std_i
            binary_exc[i] = (exc_raw[i] > threshold).astype(int)
            # thresholds_per_channel.append(threshold) # No longer needed if not plotting

        # Save raw signals
        raw_filename = data_dir / f"patient_{patient_id_str}_{condition_key.lower()}_raw.pkl"
        with open(raw_filename, 'wb') as f:
            pickle.dump(exc_raw, f)
        print(f"    Saved raw signals to: {raw_filename}")

        # Save binarized signals
        bin_filename = data_dir / f"patient_{patient_id_str}_{condition_key.lower()}_binarized.pkl"
        with open(bin_filename, 'wb') as f:
            pickle.dump(binary_exc, f)
        print(f"    Saved binarized signals to: {bin_filename}")

        patient_id_counter += 1

        # -------------------------------
        # Visualization (COMMENTED OUT)
        # -------------------------------
        # canales_a_plotear = min(5, n_channels) # Plots a maximum of 5 channels
        # plot_xlim_start = 0
        # plot_xlim_end = 2000 # Time range for detailed visualization

        # print(f"\nGenerating visualizations for condition: '{condition_name}'...")

        # # Plot 1: Original excitatory signals
        # plt.figure(figsize=(16, 10))
        # plt.title(f"{condition_params['plot_lim_title']} - Excitatory Signals (exc_ext = {condition_params['exc_ext']})", fontsize=16)
        # for i in range(canales_a_plotear):
        #     plt.plot(time, exc_raw[i], label=f"Channel {i+1}")
        # plt.xlabel("Time (ms)")
        # plt.ylabel("Activity")
        # plt.grid(True)
        # plt.legend()
        # plt.xlim(plot_xlim_start, plot_xlim_end)
        # plt.tight_layout()
        # plt.show()

        # # ---
        # # Plot 2: Combination of original signals with thresholds AND binarized signals in subplots per channel
        # # ---
        # plt.figure(figsize=(16, 12))
        # plt.suptitle(f"{condition_params['plot_lim_title']} - Original Activity, Threshold, and Binarized per Channel", fontsize=16)

        # for i in range(canales_a_plotear):
        #     # Subplot for original signal and threshold
        #     plt.subplot(canales_a_plotear, 2, 2*i + 1) # Row i, column 1
        #     plt.plot(time, exc_raw[i], label=f"Channel {i+1} Original", alpha=0.8)
        #     plt.axhline(y=thresholds_per_channel[i], color='r', linestyle=':', label=f"Threshold ({threshold_multiplier} STD)")
        #     plt.title(f"Channel {i+1} - Original + Threshold")
        #     plt.xlabel("Time (ms)")
        #     plt.ylabel("Activity")
        #     plt.grid(True)
        #     plt.legend(loc='upper right')
        #     plt.xlim(plot_xlim_start, plot_xlim_end)

        #     # Subplot for binarized signal
        #     plt.subplot(canales_a_plotear, 2, 2*i + 2) # Row i, column 2
        #     plt.plot(time, binary_exc[i], label=f"Channel {i+1} Binarized", color='purple', drawstyle='steps-post')
        #     plt.title(f"Channel {i+1} - Binarized Signal")
        #     plt.xlabel("Time (ms)")
        #     plt.ylabel("State (0/1)")
        #     plt.grid(True)
        #     plt.legend(loc='upper right')
        #     plt.xlim(plot_xlim_start, plot_xlim_end)
        #     plt.ylim(-0.1, 1.1)

        # plt.tight_layout(rect=[0, 0.03, 1, 0.96]) # Adjust layout for the main title
        # plt.show()

print(f"\n Simulation and data saving for {num_patients} patients completed. No plots generated.")

Data directory created/ensured at: /Users/diegohernandez/Documents/GitHub/Criticality_HOI/data
Loading 'hcp' dataset...
'hcp' dataset loaded with 80 channels.

--- Simulating 40 patients for condition: Healthy_Critical ---
  Simulating patient 001 (Healthy_Critical)...
    Saved raw signals to: ../data/patient_001_healthy_critical_raw.pkl
    Saved binarized signals to: ../data/patient_001_healthy_critical_binarized.pkl
  Simulating patient 002 (Healthy_Critical)...
    Saved raw signals to: ../data/patient_002_healthy_critical_raw.pkl
    Saved binarized signals to: ../data/patient_002_healthy_critical_binarized.pkl
  Simulating patient 003 (Healthy_Critical)...
    Saved raw signals to: ../data/patient_003_healthy_critical_raw.pkl
    Saved binarized signals to: ../data/patient_003_healthy_critical_binarized.pkl
  Simulating patient 004 (Healthy_Critical)...
    Saved raw signals to: ../data/patient_004_healthy_critical_raw.pkl
    Saved binarized signals to: ../data/patient_004_heal